In [7]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.gutenberg.org/browse/languages/zh'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

# 選出所有書籍連結
books = soup.select('li.pgdbetext > a[href]')

# 限制只抓前 5 本來測試（正式作業你可以設成 200）
for book in books[:300]:
    title = book.text.strip()
    link = 'https://www.gutenberg.org' + book['href']
    print(f'書名：{title}')
    print(f'連結：{link}')
    print('--------')


書名：豆棚閒話
連結：https://www.gutenberg.org/ebooks/25328
--------
書名：戲中戲
連結：https://www.gutenberg.org/ebooks/24225
--------
書名：比目魚
連結：https://www.gutenberg.org/ebooks/24185
--------
書名：比目魚
連結：https://www.gutenberg.org/ebooks/27119
--------
書名：Study of Inner Cultivation
連結：https://www.gutenberg.org/ebooks/38585
--------
書名：三字經
連結：https://www.gutenberg.org/ebooks/12479
--------
書名：山水情
連結：https://www.gutenberg.org/ebooks/25146
--------
書名：山海經
連結：https://www.gutenberg.org/ebooks/25288
--------
書名：施公案
連結：https://www.gutenberg.org/ebooks/23825
--------
書名：施公案
連結：https://www.gutenberg.org/ebooks/25393
--------
書名：易經
連結：https://www.gutenberg.org/ebooks/25501
--------
書名：木蘭奇女傳
連結：https://www.gutenberg.org/ebooks/23938
--------
書名：海公案
連結：https://www.gutenberg.org/ebooks/54494
--------
書名：燕丹子
連結：https://www.gutenberg.org/ebooks/24068
--------
書名：狄公案
連結：https://www.gutenberg.org/ebooks/27686
--------
書名：百家姓
連結：https://www.gutenberg.org/ebooks/25196
--------
書名：禮記
連結：https://www.gutenberg.org/ebooks/24048

In [8]:
import requests
from bs4 import BeautifulSoup
import re
import os

# --- 設定部分 ---
# 古騰堡計畫中文書籍列表的URL
MAIN_URL = 'https://www.gutenberg.org/browse/languages/zh'
# 儲存書籍內容的資料夾名稱
OUTPUT_FOLDER = 'project_gutenberg'

BOOK_LIMIT = 300

# 正則表達式：匹配中文字符及常用中文標點符號
# \u4e00-\u9fa5: 中文字符 (Chinese characters)
# \u3000-\u303F: CJK 符號和標點 (CJK Symbols and Punctuation, e.g., 、，。？！)
# \uff00-\uffef: 全形標點、半形和全形字符、假名等 (Halfwidth and Fullwidth Forms, e.g., ＂＃＄％＆＇（）＊＋，－．／：；＜＝＞？＠［＼］＾＿｀｛｜｝～)
CHINESE_PATTERN = re.compile(r'[\u4e00-\u9fa5\u3000-\u303F\uff00-\uffef]+')

def create_output_folder(folder_name):
    """
    創建儲存書籍內容的資料夾，如果它不存在的話。
    """
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
        print(f"已創建資料夾：'{folder_name}'")
    else:
        print(f"資料夾 '{folder_name}' 已存在。")

def clean_filename(title):
    """
    清理書籍標題，使其適合作為檔案名稱。
    移除或替換檔名中不允許的特殊字元。
    """
    # Windows 和 Linux/macOS 中常見的非法檔名字元
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    for char in invalid_chars:
        title = title.replace(char, '_')
    # 移除句點開頭或結尾的空白，並移除首尾空白
    title = title.strip().strip('.')
    return title

def get_book_content(book_url):
    """
    根據書籍內容頁面的 URL，下載並提取純中文字內容。
    """
    try:
        content_res = requests.get(book_url, timeout=10) # 增加 timeout 避免長時間等待
        content_res.raise_for_status() # 檢查 HTTP 請求是否成功 (200 OK)
        
        # 由於古騰堡的文本檔案通常是純 UTF-8，直接設定編碼
        content_res.encoding = 'utf-8' 
        content_soup = BeautifulSoup(content_res.text, 'lxml')

        book_raw_text = ""
        
        # 對於 Project Gutenberg 的 .html.images 頁面，內容通常直接在 body 或一個主 div 內
        # 最保險的方式是獲取整個 body 的 text，再進行正則篩選
        if content_soup.body:
            book_raw_text = content_soup.body.get_text(separator='\n')
        else:
            print(f'警告：未找到內容頁面 body 標籤。URL: {book_url}')
            return None

        # 使用正則表達式提取中文字及標點符號
        chinese_text_lines = []
        for line in book_raw_text.splitlines():
            # 使用 findall 會找到所有符合模式的連續字串
            # 由於我們需要保留標點，所以 `findall` 並且 `join` 是合適的
            matches = CHINESE_PATTERN.findall(line)
            if matches:
                chinese_text_lines.append("".join(matches))
        
        return "\n".join(chinese_text_lines)

    except requests.exceptions.RequestException as e:
        print(f'下載書籍內容失敗 (URL: {book_url})：{e}')
        return None
    except Exception as e:
        print(f'解析書籍內容時發生錯誤 (URL: {book_url})：{e}')
        return None

def main():
    """
    主執行函數，協調爬取和儲存書籍的流程。
    """
    create_output_folder(OUTPUT_FOLDER)

    print(f"正在從 {MAIN_URL} 獲取中文書籍列表...")
    try:
        res = requests.get(MAIN_URL, timeout=10)
        res.raise_for_status() # 檢查 HTTP 請求是否成功
        soup = BeautifulSoup(res.text, 'lxml')
    except requests.exceptions.RequestException as e:
        print(f"無法獲取主頁面列表：{e}")
        return

    # 選出所有書籍連結，這些連結會導向每本書的「資訊頁面」
    books = soup.select('li.pgdbetext > a[href]')
    
    # 應用書籍數量限制
    books_to_process = books[:BOOK_LIMIT]
    total_books_found = len(books)
    print(f"共找到 {total_books_found} 本中文書籍，將處理其中 {len(books_to_process)} 本。")
    print("這可能需要一些時間，請耐心等待...")

    for i, book in enumerate(books_to_process):
        title = book.text.strip()
        # 組合完整的書籍資訊頁面連結
        info_page_link = 'https://www.gutenberg.org' + book['href']
        
        print(f'\n--- [{i+1}/{len(books_to_process)}] {title} ---')
        print(f'資訊頁：{info_page_link}')

        try:
            # 1. 訪問書籍的「資訊頁面」
            info_page_res = requests.get(info_page_link, timeout=10)
            info_page_res.raise_for_status()
            info_page_soup = BeautifulSoup(info_page_res.text, 'lxml')

            # 2. 在書籍資訊頁面中，使用 CSS 選擇器精確尋找 "Read now!" 連結
            content_html_link_tag = info_page_soup.select_one('a.link.read_html')
            
            content_url = None
            if content_html_link_tag and 'href' in content_html_link_tag.attrs:
                content_url = 'https://www.gutenberg.org' + content_html_link_tag['href']
            
            if content_url:
                print(f'HTML 連結：{content_url}')
                
                # 3. 獲取並處理書籍內容
                cleaned_chinese_text = get_book_content(content_url)
                
                if cleaned_chinese_text:
                    # 4. 將純中文內容儲存到 .txt 檔案
                    safe_title = clean_filename(title)
                    filename = os.path.join(OUTPUT_FOLDER, f'{safe_title}.txt')
                    
                    with open(filename, 'w', encoding='utf-8') as f:
                        f.write(cleaned_chinese_text)
                    print(f' 已儲存：{filename}')
                else:
                    print(f' 未能提取 {title} 的中文內容。')
            else:
                print(f' 未找到 {title} 的內容 HTML 連結 (a.link.read_html)，跳過。')

        except requests.exceptions.RequestException as e:
            print(f' 下載 {title} 的資訊頁失敗：{e}')
        except Exception as e:
            print(f' 處理 {title} 時發生未知錯誤：{e}')
        print('----------------------------------')

if __name__ == "__main__":
    main()

資料夾 'project_gutenberg' 已存在。
正在從 https://www.gutenberg.org/browse/languages/zh 獲取中文書籍列表...
共找到 507 本中文書籍，將處理其中 300 本。
這可能需要一些時間，請耐心等待...

--- [1/300] 豆棚閒話 ---
資訊頁：https://www.gutenberg.org/ebooks/25328
HTML 連結：https://www.gutenberg.org/ebooks/25328.html.images
 已儲存：project_gutenberg\豆棚閒話.txt
----------------------------------

--- [2/300] 戲中戲 ---
資訊頁：https://www.gutenberg.org/ebooks/24225
HTML 連結：https://www.gutenberg.org/ebooks/24225.html.images
 已儲存：project_gutenberg\戲中戲.txt
----------------------------------

--- [3/300] 比目魚 ---
資訊頁：https://www.gutenberg.org/ebooks/24185
HTML 連結：https://www.gutenberg.org/ebooks/24185.html.images
 已儲存：project_gutenberg\比目魚.txt
----------------------------------

--- [4/300] 比目魚 ---
資訊頁：https://www.gutenberg.org/ebooks/27119
HTML 連結：https://www.gutenberg.org/ebooks/27119.html.images
 已儲存：project_gutenberg\比目魚.txt
----------------------------------

--- [5/300] Study of Inner Cultivation ---
資訊頁：https://www.gutenberg.org/ebooks/38585
HTML 連結：https://ww

KeyboardInterrupt: 